# Minimum Edit Distance


In [ ]:

from typing import List, Tuple, Optional

def min_edit_distance(src: str, tgt: str,
                      ins_cost: int = 1, del_cost: int = 1, sub_cost: int = 1,
                      transp_cost: Optional[int] = None) -> Tuple[int, List[Tuple[str, str, str]]]:
    
    m, n = len(src), len(tgt)
    D = [[0] * (n + 1) for _ in range(m + 1)]
    bp = [[None] * (n + 1) for _ in range(m + 1)]

    for i in range(1, m + 1):
        D[i][0] = i * del_cost
        bp[i][0] = ("del", 1)
    for j in range(1, n + 1):
        D[0][j] = j * ins_cost
        bp[0][j] = ("ins", 1)

    for i in range(1, m + 1):
        for j in range(1, n + 1):
            cost_sub = 0 if src[i - 1] == tgt[j - 1] else sub_cost

            best_cost = D[i - 1][j - 1] + cost_sub
            best_bp = ("match" if cost_sub == 0 else "sub", 1, 1)

            ins_c = D[i][j - 1] + ins_cost
            if ins_c < best_cost:
                best_cost = ins_c
                best_bp = ("ins", 0, 1)

            del_c = D[i - 1][j] + del_cost
            if del_c < best_cost:
                best_cost = del_c
                best_bp = ("del", 1, 0)

            if transp_cost is not None and i >= 2 and j >= 2:
                if src[i - 1] == tgt[j - 2] and src[i - 2] == tgt[j - 1]:
                    transp_c = D[i - 2][j - 2] + transp_cost
                    if transp_c < best_cost:
                        best_cost = transp_c
                        best_bp = ("transp", 2, 2)

            D[i][j] = best_cost
            bp[i][j] = best_bp

    ops: List[Tuple[str, str, str]] = []
    i, j = m, n
    while i > 0 or j > 0:
        move = bp[i][j]
        if move is None:
            break
        op = move[0]
        if len(move) == 3:
            di, dj = move[1], move[2]
        else:
            di, dj = move[1], 0

        if op == "match":
            ops.append(("match", src[i - 1], tgt[j - 1]))
        elif op == "sub":
            ops.append(("sub", src[i - 1], tgt[j - 1]))
        elif op == "ins":
            ops.append(("ins", "-", tgt[j - 1]))
        elif op == "del":
            ops.append(("del", src[i - 1], "-"))
        elif op == "transp":
            ops.append(("transp", src[i - 2:i], tgt[j - 2:j]))
        else:
            raise RuntimeError(f"Unknown op: {op}")

        i -= di
        j -= dj

    ops.reverse()
    return D[m][n], ops


In [ ]:

def pretty_alignment(src: str, tgt: str, ops: List[Tuple[str,str,str]]) -> str:
    """Красивый вывод выравнивания и операций."""
    top, mid, bot = [], [], []
    for op, a, b in ops:
        if op == "transp":
            top.append(a)
            bot.append(b)
            mid.append("⟂" if a != b else "|")
        else:
            top.append(a if a != "-" else "-")
            bot.append(b if b != "-" else "-")
            mid.append("|" if op == "match" else ("*" if op == "sub" else "·"))

    s_top = " ".join(top)
    s_mid = " ".join(mid)
    s_bot = " ".join(bot)

    op_lines = ["Operations:"]
    for k, (op, a, b) in enumerate(ops, 1):
        op_lines.append(f"{k:02d}. {op:6s}  {a!r:>6} → {b!r}")

    return s_top + "\n" + s_mid + "\n" + s_bot + "\n\n" + "\n".join(op_lines)


In [ ]:

cases = [
    ("kitten", "sitting",   dict(ins_cost=1, del_cost=1, sub_cost=1, transp_cost=None)),
    ("intention", "execution", dict(ins_cost=1, del_cost=1, sub_cost=1, transp_cost=None)),
    ("abcd", "acbd", dict(ins_cost=1, del_cost=1, sub_cost=1, transp_cost=1)),  # транспозиция = 1
    ("distance", "editing", dict(ins_cost=1, del_cost=1, sub_cost=1, transp_cost=1)),
]

for s, t, kwargs in cases:
    d, ops = min_edit_distance(s, t, **kwargs)
    print(f"{s!r} → {t!r} | distance = {d}")
    print(pretty_alignment(s, t, ops))
    print("-" * 64)
